In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from trl import SFTTrainer
from jinja2 import Template
import yaml
import torch

In [13]:
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

NEW_MODEL_NAME = "TinyButMighty"

DATASET_NAME = "macadeliccc/opus_samantha"

SPLIT = "train"

MAX_SEQ_LENGTH = 2048

num_train_epochs = 1

license = "apache-2.0"

username = "fahdmirzac"

learning_rate = 1.41e-5

per_device_train_batch_size = 4

gradient_accumulation_steps = 1

In [14]:
device = "cpu"
if torch.backends.mps.is_built():
    device = torch.device("mps")

device

device(type='mps')

[codecarbon INFO @ 11:02:18] Energy consumed for RAM : 0.000625 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:02:18] Energy consumed for all CPUs : 0.004427 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:02:18] 0.005052 kWh of electricity used since the beginning.


In [15]:
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, trust_remote_code=True)
#model.to(device)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
[codecarbon INFO @ 11:02:33] Energy consumed for RAM : 0.000650 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:02:33] Energy consumed for all CPUs : 0.004604 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:02:33] 0.005254 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:02:48] Energy consumed for RAM : 0.000675 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:02:48] Energy consumed for all CPUs : 0.004781 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:02:48] 0.005456 kWh of electricity used since the beginning.
Loading checkpoint shards: 100%|██████████| 2/2 [00:35<00:00, 17.66s/it]


[codecarbon INFO @ 11:03:03] Energy consumed for RAM : 0.000700 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:03:03] Energy consumed for all CPUs : 0.004959 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:03:03] 0.005659 kWh of electricity used since the beginning.


In [16]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# username = "base"
# model.save_pretrained(f"{username}/{NEW_MODEL_NAME}")

# tokenizer.save_pretrained(f"{username}/{NEW_MODEL_NAME}")



In [5]:
from datasets import load_dataset
dataset = load_dataset("macadeliccc/opus_samantha", split="train[:5%]")


In [6]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 319
})

In [17]:
dataset = load_dataset("macadeliccc/opus_samantha", split="train[:1%]")
# total_dataset_size = len(dataset)
# dataset = load_dataset("macadeliccc/opus_samantha", split="train", skip=int(0.9 * total_dataset_size), take=int(0.1 * total_dataset_size))



EOS_TOKEN=tokenizer.eos_token_id



def process_dataset(mydata):

    conversations = mydata["conversations"]

    texts = []

    mapper = {"system": "system\n", "human": "\nuser\n", "gpt": "\nassistant\n"}

    end_mapper = {"system": "", "human": "", "gpt": ""}

    for c in conversations:

        text = "".join(f"{mapper[(turn := x['from'])]} {x['value']}\n{end_mapper[turn]}" for x in c)

        texts.append(f"{text}{EOS_TOKEN}")

    return {"text": texts}



dataset = dataset.map(process_dataset, batched=True)

print(dataset['text'][2])
print("-------------")
print(len(dataset))
# print(len(dataset['text'][:100]))
# dataset = {"text": dataset['text'][:100]}
# print(len(dataset['text']))


[codecarbon INFO @ 11:03:18] Energy consumed for RAM : 0.000725 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:03:18] Energy consumed for all CPUs : 0.005136 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:03:18] 0.005861 kWh of electricity used since the beginning.
Map: 100%|██████████| 32/32 [00:00<00:00, 1239.93 examples/s]


user
 I'm really nervous about this upcoming presentation at work. Public speaking just isn't my thing, and I'm worried I'll mess it up. How can I calm my nerves and do well?

assistant
 First of all, it's totally normal to feel nervous about public speaking. Even the most seasoned speakers get butterflies sometimes. The key is to channel that nervous energy into excitement and preparation.

user
 That's a good point. I have been preparing a lot, practicing my slides and my talking points. But in the moment, I'm afraid my mind will just go blank.

assistant
 I totally get that fear. One thing that can help is to focus on your breathing. Take slow, deep breaths before you start and during your presentation if you need to center yourself. It's also helpful to remember that your audience is rooting for you. They want to see you succeed!

user
 You're right, they're not out to get me. I think I also put a lot of pressure on myself to be perfect. But realistically, a minor slip-up isn't th

[codecarbon INFO @ 11:03:33] Energy consumed for RAM : 0.000750 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:03:33] Energy consumed for all CPUs : 0.005313 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:03:33] 0.006063 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:04:03] Energy consumed for RAM : 0.000800 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:04:03] Energy consumed for all CPUs : 0.005667 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:04:03] 0.006467 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:04:29] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:04:29] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:04:29] 0.000404 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:04:48] Energy consumed for RAM : 0.000875 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:04:48] Energy consumed for all CPUs : 0.006198 kWh. Total CPU Power : 42.5 W
[codeca

In [18]:
args = TrainingArguments(

    per_device_train_batch_size=1,

    gradient_accumulation_steps=gradient_accumulation_steps,

    gradient_checkpointing=True,

    learning_rate=2e-5,

    lr_scheduler_type="cosine",

    max_steps=-1,

    num_train_epochs=num_train_epochs,

    save_strategy="no",

    logging_steps=1,

    output_dir=NEW_MODEL_NAME,

    optim="paged_adamw_32bit",

    bf16=True,
    no_cuda=True,
    #use_mps_device=True
)

/Users/deepakmishra/.pyenv/versions/3.10.14/envs/denv/lib/python3.10/site-packages/transformers/training_args.py:1509: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


[codecarbon INFO @ 11:03:48] Energy consumed for RAM : 0.000775 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 11:03:48] Energy consumed for all CPUs : 0.005490 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:03:48] 0.006265 kWh of electricity used since the beginning.


In [19]:
trainer = SFTTrainer(

    model=model,

    args=args,

    train_dataset=dataset,

    dataset_text_field="text",

    max_seq_length=MAX_SEQ_LENGTH,

    formatting_func=process_dataset, 
    

)
print("TRAINER LOADED")


/Users/deepakmishra/.pyenv/versions/3.10.14/envs/denv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/deepakmishra/.pyenv/versions/3.10.14/envs/denv/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/deepakmishra/.pyenv/versions/3.10.14/envs/denv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you 

TRAINER LOADED


In [20]:
print('TRAINING STARTED---------------')
trainer.train()
print("MODEL TRAINED......")

TRAINING STARTED---------------


/Users/deepakmishra/.pyenv/versions/3.10.14/envs/denv/lib/python3.10/site-packages/transformers/trainer.py:3276: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/Users/deepakmishra/.pyenv/versions/3.10.14/envs/denv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
You are not running the flash-attention implementation, expect numerical differences.
[codecarbon INFO @ 11:04:14] Energy consumed for RAM : 0.000025 kWh. RAM P